In [1]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [3]:
train, dev = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
train.shape, dev.shape

((960, 8), (240, 8))

In [5]:
df['rating'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA+', 'BB+', 'BB-', 'A-', 'A+', 'B',
       'AA-', 'BBB+', 'BBB-', 'B-', 'AA', 'B+', 'C'], dtype=object)

In [6]:
categories = ['C', 'BBB', 'BBB+', 'BBB-', 'BB', 'BB-', 'BB+']

In [7]:
dev = dev.query('rating in @categories')

In [8]:
train = train.query('rating in @categories')

In [9]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [10]:
nlp = spacy.load('ru_core_news_lg')

In [11]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [12]:
create_docbin(train).to_disk("train.spacy")
create_docbin(dev).to_disk("dev.spacy")

In [13]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       53.33    0.53
  0     200          24.63       63.84    0.64
  1     400          17.07       68.61    0.69
  1     600          10.28       69.68    0.70
  2     800           7.90       73.95    0.74
  2    1000           5.35       72.48    0.72
  3    1200           2.24       75.06    0.75
  4    1400           2.54       76.17    0.76
  4    1600           1.68       75.50    0.75
  5    1800           1.63       73.97    0.74
  5    2000           1.21       76.31    0.76
  6    2200           0.52       75.48    0.75
  7    2400           0.95       75.46    0